### **Récupereration le file et lecture du fichier**

In [62]:
#importer les packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [63]:
#lire
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_u6lujuX_CVtuZ9i.csv')

FileNotFoundError: ignored

In [ ]:
pd.set_option('display.max_rows',df.shape[0]+1)

In [ ]:
pd.set_option('display.max_rows',10)


In [ ]:
#les valeurs manquantes
df.info()
df

In [ ]:
df.describe(include=['O'])

In [ ]:
df.isnull().sum().sort_values(ascending=True)

In [ ]:
#renseigner les valeurs
cat_data = []
num_data = []
for i,c in enumerate(df.dtypes):
  if c==object:
    cat_data.append(df.iloc[:,i])
  else:
      num_data.append(df.iloc[:,i])
cat_data = pd.DataFrame(cat_data).transpose()
num_data = pd.DataFrame(num_data).transpose()

In [ ]:
#pour les vzriables categoriques on va renplacer les valeurs manquantes pare les valeurs repetitives
cat_data = cat_data.apply(lambda x:x.fillna(x.value_counts().index[0]))
cat_data.isnull().sum().any()


In [ ]:
#pour les vzriables categoriques on va renplacer les valeurs manquantes pare les valeurs repetitives
num_data.fillna(method='bfill',inplace=True)
num_data.isnull().sum().any()


In [ ]:
target_value = {'Y': 1,'N': 0}
target = cat_data['Loan_Status']
cat_data.drop('Loan_Status',axis=1,inplace=True)
target = target.map(target_value)
target

In [ ]:
le=LabelEncoder()
for i in cat_data:
  cat_data[i]=le.fit_transform(cat_data[i])
cat_data

In [ ]:
#Supprimer loan_id
cat_data.drop('Loan_ID',axis=1,inplace=True)
cat_data

In [ ]:
#concatener cat_data et spécifier la colonne target
X=pd.concat([cat_data,num_data],axis=1)
y = target
target.value_counts()
X

In [ ]:
#bases de données utilisées pour l'analyses opératoitrs 
df=pd.concat([cat_data,num_data,target],axis=1)
df

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(target)
yes = target.value_counts()[0]/len(target)
no = target.value_counts()[1]/len(target)
print(f'le pourecentage des crédits accord est: {yes}')
print(f'le pourecentage des crédits non ccord est: {no}')
print(f'la cardinalité non accordé: {target.value_counts()[0]}')
print(f'la cardinalité: accordé {target.value_counts()[1]}')

In [ ]:
#la champ de gens dont la demande est non agréée 0 et  agréée 1
grid = sns.FacetGrid(df,col='Loan_Status',size=3.2,aspect=1.6)
grid.map(sns.countplot,'Credit_History')

In [ ]:
#sexe
grid = sns.FacetGrid(df,col='Loan_Status',size=3.2,aspect=1.6)
grid.map(sns.countplot,'Gender')

In [ ]:
#Married
grid = sns.FacetGrid(df,col='Loan_Status',size=3.2,aspect=1.6)
grid.map(sns.countplot,'Married')

In [ ]:
#Education
grid = sns.FacetGrid(df,col='Loan_Status',size=3.2,aspect=1.6)
grid.map(sns.countplot,'Education')

In [ ]:
#revenu du demandeur
plt.scatter(df['CoapplicantIncome'],df['Loan_Status'])

In [ ]:
df.groupby('Loan_Status').median()

In [ ]:
#diviser la base de données en une base de données test et d'entrainement
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train,test in sss.split(X,y):
  X_train,X_test = X.iloc[train],X.iloc[test]
  y_train,y_test = y.iloc[train],y.iloc[test]

print('X_train taille: ', X_train.shape)
print('X_test taille: ', X_test.shape)
print('X_train taille: ', y_train.shape)
print('X_test taille: ', y_test.shape)

In [ ]:
#On va appliquer les 3 algorith => logistic regression, KNN, Decisison Tree
models = {
    'LogisticRegression': LogisticRegression(random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=1, random_state=42)
}

# La fonction de précision
def accu(y_true,y_pred,retu=False):
  acc=accuracy_score(y_true,y_pred)
  if retu:
    return acc
  else:
    print(f'la précision du modèle est: {acc}')

  #c'est la fonction d'application des modèles
def train_test_eval(models,X_train,y_train,X_test,y_test):
  for name,model in models.items():
    print(name,': ')
    model.fit(X_train,y_train)
    accu(y_test,model.predict(X_test))
    print('_'*30)
train_test_eval(models,X_train,y_train,X_test,y_test)

In [ ]:
X_2=X[['Credit_History','Married','CoapplicantIncome']];



In [ ]:
#diviser la base de données en une base de données test et d'entrainement
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train,test in sss.split(X_2,y):
  X_train,X_test = X_2.iloc[train],X_2.iloc[test]
  y_train,y_test = y.iloc[train],y.iloc[test]

print('X_train taille: ', X_train.shape)
print('X_test taille: ', X_test.shape)
print('X_train taille: ', y_train.shape)

In [ ]:
train_test_eval(models,X_train,y_train,X_test,y_test)

In [ ]:
#Appliquer la logistic regression sur notre base de donnée
Classifier=LogisticRegression()
Classifier.fit(X_2,y)

In [ ]:
#Enregister mon modele
pickle.dump(Classifier,open('model.pkl','wb'))